In [1]:
#pip install lightgbm


In [2]:
import joblib
import pandas as pd
import threading
import lightgbm as lgb
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

In [ ]:
data = pd.read_csv("DatasetInitalComplete.csv", low_memory=False)
data = data.drop(["7","10","11","13","14"],axis = 1)

In [ ]:
data.shape

In [ ]:
data

In [ ]:
data.columns

In [ ]:
non_numerical_columns = data.select_dtypes(exclude=['number']).columns

# Convert to a list if needed
non_numerical_columns_list = list(non_numerical_columns)

print(non_numerical_columns_list)

In [ ]:
from sklearn import preprocessing
le = preprocessing.LabelEncoder()

In [ ]:
from sklearn import preprocessing
le = preprocessing.LabelEncoder()

In [ ]:
import pickle

data_new = data
data1 = {}
save_file = open("cases.dat", "wb")

for j in non_numerical_columns_list:
    a = list(set(list(data_new[j])))
    index = 0
    dic = {}
    for i in a:
        dic[i] = index
        index += 1

    data1[j] = dic

pickle.dump(data1, save_file)
save_file.close()

In [ ]:
data_file = open("cases.dat", "rb")
cases = pickle.load(data_file)
type(cases)
data_file.close()

In [ ]:
for i in cases:
    for j in cases[i]:
        data_new[i] = data_new[i].replace(j, cases[i][j])
        

In [ ]:
classes = []
for column in non_numerical_columns_list:
    data[column] = le.fit_transform(data[column])
    print(le.classes_)
    classes.append(le.classes_)

In [ ]:
import pickle
data_file = open("cases.dat", "rb")
cases = pickle.load(data_file)
type(cases)
data_file.close()

In [ ]:
for i in cases:
    for j in cases[i]:
        data_new[i] = data_new[i].replace(j, cases[i][j])

In [ ]:
data_new

In [ ]:
df_sorted = data.sort_values(by='target',ascending = False)
df_target_1 = data[data['target'] == 1]
df_target_0 = data[data['target'] == 0]

#df_target_0[:392]
batch1 = pd.concat([df_target_1, df_target_0[:]], ignore_index=True)
batch1


In [ ]:
#pip install matplotlib

In [ ]:
correlation_matrix = batch1.corr()
import matplotlib.pyplot as plt
plt.matshow(correlation_matrix, cmap='viridis')
plt.show()


In [ ]:
correlated_pairs = []
threshold = 0.95
for i in range(len(correlation_matrix.columns)):
    for j in range(i):
        if ((correlation_matrix.iloc[i, j]) > threshold):
            colname1 = correlation_matrix.columns[i]
            colname2 = correlation_matrix.columns[j]
            correlated_pairs.append((colname1, colname2, correlation_matrix.iloc[i, j]))

# Print the correlated pairs
print("Pairs of columns with correlation above threshold:")
print(len(correlated_pairs))
for pair in correlated_pairs:
    print(f"Columns: {pair[0]} and {pair[1]} with correlation: {pair[2]:.2f}")

In [ ]:
train_set, test_set = train_test_split(batch1, test_size=0.1, stratify=batch1['target'], random_state=42)
scaler = StandardScaler()
X_train = train_set.drop(columns=['target'])
y_train = train_set['target']
X_test = test_set.drop(columns=['target'])
y_test = test_set['target']
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [ ]:
from sklearn.metrics import accuracy_score, confusion_matrix

In [ ]:
tprsacc = []
final_predict_data = data.drop(["target"],axis=1)

def trainer(number,batch):
    #print(pd.DataFrame(batch))
    print(number)
    scaler = StandardScaler()

    train_set, test_set = train_test_split(batch, test_size=0.2, stratify=batch['target'], random_state=42)
    X_train = train_set.drop(columns=['target'])
    y_train = train_set['target']
    X_test = test_set.drop(columns=['target'])
    y_test = test_set['target']
    X_train = scaler.fit_transform(X_train)
    X_test = scaler.transform(X_test)
    
    model = lgb.LGBMClassifier(verbose=-1)
    model.fit(X_train, y_train)
    #print("model"+str(number))
    name = "model" + str(number) + ".joblib"
    
    predictions = model.predict(X_test)
    tn, fp, fn, tp = confusion_matrix(y_test, predictions).ravel()
    accuracy = accuracy_score(y_test,predictions)
    tprsacc.append((tp/(tp+fn),accuracy))

    joblib.dump(model, name)
    
    #predictions = model.predict(X_test)
    #tn, fp, fn, tp = confusion_matrix(y_test, predictions).ravel()
    
    #accuracy = accuracy_score(y_test,predictions)
    #print("model number"+str(i)+" true positives: "+str(tp/(tp+fn)))
    #print("model number"+str(i)+" accuracu: "+str(accuracy))
    
    
    # Make predictions and evaluate
    
    

In [ ]:
threads = []
import time

start_time = time.time()

for i in range(0,1018):
    batch = batch1 = pd.concat([df_target_1, df_target_0[i*392:(i*392)+392]], ignore_index=True)
       
    thread = threading.Thread(target=trainer, args=(i,batch))
    threads.append(thread)
    thread.start()
end_time = time.time()
total_time = end_time - start_time
total_time
#batch

In [ ]:
tprsacc 

In [ ]:
def st(k):
    return k[1]
trial = tprsacc.sort(key =st )
print(trial)